In [29]:
from bs4 import BeautifulSoup
import requests

In [30]:
def pages_to_scrape(pages):
    Lista_url= []
    for page in range(1,pages+1):

        search_url = f'https://www.domiporta.pl/mieszkanie/sprzedam/mazowieckie/warszawa?PageNumber={pages}'
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # Znajdź wszystkie elementy <article> o klasie "sneakpeak"
        sneakpeak_articles = soup.find_all('article', class_='sneakpeak')

# Przetwarzanie każdego elementu sneakpeak
        for sneakpeak_article in sneakpeak_articles:
            # Znajdź element <a> wewnątrz każdego elementu sneakpeak i pobierz wartość atrybutu "href"
            href_value = sneakpeak_article.find('a')['href']
            Lista_url.append(href_value)
    print('Skończone pobieranie ogłoszeń')        
    return Lista_url

In [31]:
x = pages_to_scrape(2)

/nieruchomosci/sprzedam-mieszkanie-dwupokojowe-warszawa-praga-polnoc-karola-szymanowskiego-49m2/154637291
/nowe/ogloszenie/ursus_centralny_warszawa-ursus_644093
/nowe/ogloszenie/jutrzenki_92_warszawa-wlochy_643266
/nowe/ogloszenie/jutrzenki_92_warszawa-wlochy_643275
/nowe/ogloszenie/jutrzenki_92_warszawa-wlochy_643286
/nowe/ogloszenie/jutrzenki_92_warszawa-wlochy_634511
/nowe/ogloszenie/jutrzenki_92_warszawa-wlochy_634455
/nowe/ogloszenie/jutrzenki_92_warszawa-wlochy_634469
/nowe/ogloszenie/jutrzenki_92_warszawa-wlochy_634436
/nowe/ogloszenie/miasto_moje_warszawa-bialoleka_643707
/nowe/ogloszenie/przystanek_tarchomin_warszawa-bialoleka_643806
/nowe/ogloszenie/przystanek_tarchomin_warszawa-bialoleka_643802
/nowe/ogloszenie/miasteczko_nova_ochota_warszawa-wlochy_634199
/nowe/ogloszenie/miasteczko_nova_ochota_warszawa-wlochy_615023
/nowe/ogloszenie/nu-_warszawa_wola_-_pierwszy_etap_inwestycji_warszawa-wola_613267
/nieruchomosci/sprzedam-mieszkanie-czteropokojowe-warszawa-wilanow-sarmacka-